In [ ]:
!git clone https://github.com/derInformatiker/AIcrowd-AIBlitz7-Solution.git
!pip install -r AIcrowd-AIBlitz7-Solution/challenge1/requirements.txt
!pip install aicrowd-cli==0.1

Cloning into 'AIcrowd-AIBlitz7-Solution'...
remote: warning: multi-pack bitmap is missing required reverse index
remote: Enumerating objects: 329, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 329 (delta 194), reused 218 (delta 121), pack-reused 0
Receiving objects: 100% (329/329), 479.09 KiB | 3.30 MiB/s, done.
Resolving deltas: 100% (194/194), done.
     |████████████████████████████████| 10.1MB 5.8MB/s 
     |████████████████████████████████| 28.2MB 108kB/s 
     |████████████████████████████████| 829kB 34.8MB/s 
     |████████████████████████████████| 122kB 41.2MB/s 
     |████████████████████████████████| 829kB 32.6MB/s 
     |████████████████████████████████| 276kB 35.3MB/s 
     |████████████████████████████████| 112kB 36.6MB/s 
     |████████████████████████████████| 952kB 33.7MB/s 
     |████████████████████████████████| 1.3MB 34.3MB/s 
     |████████████████████████████████| 143kB 43.1MB/s 
     |███████

     |████████████████████████████████| 40kB 1.7MB/s 
     |████████████████████████████████| 163kB 6.2MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 204kB 7.1MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for aicrowd-cli: filename=aicrowd_cli-0.1.0-cp37-none-any.whl size=34929 sha256=89c807e406ff0cbce43118c0af5910437a2a71d6e0f71861b65f54c84ccd303b
  Stored in directory: /root/.cache/pip/wheels/7e/ec/cc/a38391f23ad9bd044c91cd5e6e0a30f2a4265a1eb5adedf9f5
Successfully built aicrowd-cli


In [ ]:
#RESTART RUNTIME TO USE NEW PACKAGES

In [ ]:
API_KEY = "b46152c118edae059e08ebd1f8ada78f"  # Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [ ]:
!aicrowd dataset download --challenge rover-classification

!rm -rf data
!mkdir data

!unzip -q train.zip  -d data/train
!unzip -q val.zip -d data/val
!unzip -q test.zip  -d data/test

!mv train.csv data/train.csv
!mv val.csv data/val.csv
!mv sample_submission.csv data/sample_submission.csv

sample_submission.csv: 100% 164k/164k [00:00<00:00, 396kB/s]
test.zip: 100% 66.5M/66.5M [00:10<00:00, 6.31MB/s]
train.csv: 100% 689k/689k [00:00<00:00, 968kB/s]
train.zip: 100% 266M/266M [00:21<00:00, 12.2MB/s]
val.csv: 100% 65.0k/65.0k [00:00<00:00, 282kB/s]
val.zip: 100% 26.5M/26.5M [00:03<00:00, 8.70MB/s]


In [ ]:
import shutil

shutil.copy('AIcrowd-AIBlitz7-Solution/challenge1/model.py','model.py')
shutil.copy('AIcrowd-AIBlitz7-Solution/challenge1/dataset.py','dataset.py')

'dataset.py'

In [ ]:
import model
import dataset

import albumentations as A
from albumentations.augmentations.transforms import Flip

import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [ ]:
if __name__ == '__main__':
    trainer = Trainer(max_epochs = 6, gpus = 1, precision=16, amp_level='O1',deterministic=True)
    
    train_tr = A.Compose([
        A.CenterCrop(200,200,always_apply=True),
        Flip()
    ])
    
    val_tr = A.Compose([
        A.CenterCrop(200,200,always_apply=True)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':64,'train_tr':train_tr,'val_tr':val_tr})
    
    trainer.fit(model)
    trainer.test(model)
    out = trainer.predict(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth



Loaded pretrained weights for efficientnet-b3



  | Name   | Type         | Params
----------------------------------------
0 | resnet | EfficientNet | 10.7 M
----------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.797    Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


/content/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(x)



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1': 0.9977084994316101, 'test_loss': 0.009767717681825161}
--------------------------------------------------------------------------------


In [ ]:
import pandas as pd
def writeSub(p):
    labelmap = {0:'perseverance',1:'curiosity'}
    test_df = pd.read_csv('data/sample_submission.csv')
    output_list = p.int().tolist()
    output_list = [labelmap[i] for i in output_list]
    test_df['label'] = output_list
    test_df.to_csv(path_or_buf='data/submission.csv',index = False)

In [ ]:
output = torch.tensor([])

for i in range(len(out)):
    output = torch.cat((output,torch.tensor(out[i][1]).argmax(1)))

In [ ]:
output

tensor([0., 0., 1.,  ..., 0., 0., 0.])

In [ ]:
test_df = pd.read_csv('data/sample_submission.csv')
labelmap = {0:'perseverance',1:'curiosity'}
output_list = output.int().tolist()
output_list = [labelmap[i] for i in output_list]
test_df['label'] = output_list
# test_df.to_csv('submission.csv',index = False)
test_df

,ImageID,label
0,0,perseverance
1,1,perseverance
2,2,curiosity
3,3,perseverance
4,4,perseverance
...,...,...
9995,9995,perseverance
9996,9996,perseverance
9997,9997,perseverance
9998,9998,perseverance


In [ ]:
!pip install -U pandas

     |████████████████████████████████| 9.9MB 5.5MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.3 which is incompatible.
  Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5


In [ ]:
import pandas as pd
test_df.to_csv('submission.csv', index=False)

NameError: ignored

In [ ]:
writeSub(output)

TypeError: ignored

In [ ]:
!aicrowd submission create -c rover-classification -f data/submission.csv

Submission file data/submission.csv does not exist
